# Benchmarking tflite models  

This notebook parses the benchmarking.txt files generated by: 

 [https://www.tensorflow.org/lite/performance/measurement](https://www.tensorflow.org/lite/performance/measurement)  

 TensorFlow Lite benchmark tools currently measure and calculate statistics for the following important performance metrics:

- Initialization time
- Inference time of warmup state
- Inference time of steady state
- Memory usage during initialization time
- Overall memory usage

In [1]:
import os, sys, math, datetime
import pathlib
from pathlib import Path

# import workbench.config.config
from workbench.config.config import initialize
from workbench.utils.utils import create_filepaths

In [2]:
import re
from matplotlib import pyplot as plt
#import plotly.express as px
import pandas as pd


# enable plotly in VS Studio Code
#import plotly.io as pio
#pio.renderers.default = "notebook_connected"
#pio.renderers.default = "plotly_mimetype+notebook"

import wandb

In [3]:
# Configure pandas to show all columns & rows
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [4]:
models_dir = initialize()

In [5]:
automated = False

global model_name
#model_name = "efficientNetB0_1_96_c1_o3_keras"
#model_name = "mobilenetv2_0.1_96_c1_o3_l1"
model_name = "mobilenetv1_0.1_96_c3_o2_l5.MV1"
#model_name = "shufflenetv2tiny_0.1_96_c3_o3_f4l1024"

models_path, models_summary_path, models_image_path, models_layer_df_path, models_tf_path, models_tflite_path, models_tflite_opt_path = create_filepaths(model_name)

i:\tinyml\tiny_cnn\models


In [6]:
models_benchmark_path = models_dir.joinpath(model_name, f"{model_name}_benchmark.txt")
models_benchmark_path

WindowsPath('i:/tinyml/tiny_cnn/models/mobilenetv1_0.1_96_c3_o2_l5.MV1/mobilenetv1_0.1_96_c3_o2_l5.MV1_benchmark.txt')

# Helper functions

In [7]:
def clean_model_summary(filepath): 

    if not filepath.is_file():
        print(f"This file does not exist: {filepath}")
        return None

    else:
        clean_lines = []
        # Parse the MLTK model summary to grab important metrics   
        with open(filepath, "r", encoding="latin-1") as f:
            lines = f.readlines() # list containing lines of file
            for line in lines:
                line = line.strip() # remove leading/trailing white spaces
                if len(line)> 0:
                    
                    clean_lines.append(line)
                else:
                    pass
            #columns = [] # To store column names
        return clean_lines

In [8]:
def clean_column_names(df):
    cols = df.columns

    clean_cols = []
    for col in cols:
        col = col.strip()
        col = col.replace("[" , "")
        col = col.replace("]" , "")   
        clean_cols.append(col)
        
    return clean_cols

In [9]:
def string_percent_to_float(x):
    x = str(x).strip("%")
    return float(x)

In [10]:
def remove_tabs(text):
    """removes tabs from a list of strings

    Args:
        text (list(str)): list of strings that contains tabs

    Returns:
        list(str): list of strings without tabs
    """
    split_text= []
    for l in text:
        split_text.append((l.split("\t")))
        
    return split_text

# Parsing the model file

if models_benchmark_path.is_file():
    

In [11]:
lines = clean_model_summary(models_benchmark_path)
lines

['STARTING!',
 'Log parameter values verbosely: [0]',
 'Num threads: [1]',
 'Graph: [/mnt/i/tinyml/tiny_cnn/models/mobilenetv1_0.1_96_c3_o2_l5.MV1/mobilenetv1_0.1_96_c3_o2_l5.MV1_INT8.tflite]',
 'Enable op profiling: [1]',
 '#threads used for CPU inference: [1]',
 'Loaded model /mnt/i/tinyml/tiny_cnn/models/mobilenetv1_0.1_96_c3_o2_l5.MV1/mobilenetv1_0.1_96_c3_o2_l5.MV1_INT8.tflite',
 'The input model file size (MB): 0.08908',
 'Initialized session in 95.124ms.',
 'Running benchmark for at least 1 iterations and at least 0.5 seconds but terminate if exceeding 150 seconds.',
 'count=751 first=5653 curr=617 min=597 max=5653 avg=663.688 std=195',
 'Running benchmark for at least 50 iterations and at least 1 seconds but terminate if exceeding 150 seconds.',
 'count=1339 first=688 curr=670 min=650 max=1292 avg=696.187 std=69',
 'Inference timings in us: Init: 95124, First inference: 5653, Warmup (avg): 663.688, Inference (avg): 696.187',
 'Note: as the benchmark tool itself affects memory f

In [12]:
def get_profiling_lines(lines):
    #print(lines)
    # find section split
    for i, line in enumerate(lines):
        #print(f"{i} : {line}")
    #if line.str.contains('= Run Order ='):
        if "Operator-wise Profiling " in line:
            split_line = i
            #print(f"Splitting text at line {split_line}")
            model_profiling = lines[:split_line]
            operator_profiling = lines[split_line+1:]

        else:
            pass
    if i==0:     
        print("WARNING: Could not find the text: 'Operator-wise Profiling Info for Regular Benchmark Runs:'")
        return (None, None)
    else:
        return model_profiling, operator_profiling


In [13]:
model_profiling, operator_profiling = get_profiling_lines(lines)

In [14]:
def get_index_of_model_profiling_lines(lines):
    model_lines_dict = {}
    for i, line in enumerate(lines):
        if "= Run Order =" in line:
            model_lines_dict["run_order"] = i
        elif "= Top by Computation Time =" in line:
            model_lines_dict["top_by_computation_time"] = i
        elif "The input model file size" in line:
            model_lines_dict["model_file_size"] = i
        elif "Initialized session in" in line:
            model_lines_dict["initialization_ms"] = i
        elif "Inference timings in us:" in line:
            model_lines_dict["inference_timings"] = i

    #print(model_lines_dict)
    return model_lines_dict


In [15]:
model_lines_dict = get_index_of_model_profiling_lines(model_profiling)

In [16]:
def get_model_profiling_stats(model_profiling):
    model_lines_dict = get_index_of_model_profiling_lines(model_profiling)

    model_stats = {}

    # extract model size
    model_size_string = model_profiling[model_lines_dict["model_file_size"]]
    model_size_string
    model_size= float(model_size_string.split(":")[-1].strip())
    model_stats["model_size_MB"] = model_size

    # extract inference times
    inference_string = model_profiling[model_lines_dict["inference_timings"]]
    #print(inference_string)
    inference_strings = inference_string.split(",")
    for item in inference_strings:
        item_time = item.split()[-1]
        #print(item_time)

    model_stats["init_us"] = int(inference_strings[0].split()[-1].strip())
    model_stats["first_inference_us"] = int(inference_strings[1].split()[-1].strip())
    model_stats["warmup_avg_us"] = float(inference_strings[2].split()[-1].strip())
    model_stats["inference_avg_us"] = float(inference_strings[3].split()[-1].strip())

    # model initialization timings
    init_string = model_profiling[model_lines_dict["initialization_ms"]]
    model_stats["initialization_ms"] = float(init_string.split()[-1].replace("ms.",""))


    # initialization run order
    init_run_order = model_profiling[model_lines_dict["run_order"]: model_lines_dict["top_by_computation_time"]]
    init_run_order = remove_tabs(init_run_order)

    model_stats["modify_graph_with_delegate_ms_first"] = float(init_run_order[2][1].strip())
    model_stats["modify_graph_with_delegate_ms_avg"] = float(init_run_order[2][2].strip())
    model_stats["modify_graph_with_delegate_ms_%"] = float(init_run_order[2][3].strip().replace("%", ""))
    model_stats["modify_graph_with_delegate_mem_KB"] = float(init_run_order[2][5].strip())

    model_stats["allocate_tensors_ms_first"] = float(init_run_order[3][1].strip())
    model_stats["allocate_tensors_ms_avg"] = float(init_run_order[3][2].strip())
    model_stats["allocate_tensors_ms_%"] = float(init_run_order[3][3].strip().replace("%", ""))

    return model_stats

In [17]:
model_stats = get_model_profiling_stats(model_profiling)
model_stats

{'model_size_MB': 0.08908,
 'init_us': 95124,
 'first_inference_us': 5653,
 'warmup_avg_us': 663.688,
 'inference_avg_us': 696.187,
 'initialization_ms': 95.124,
 'modify_graph_with_delegate_ms_first': 0.201,
 'modify_graph_with_delegate_ms_avg': 0.201,
 'modify_graph_with_delegate_ms_%': 65.472,
 'modify_graph_with_delegate_mem_KB': 0.0,
 'allocate_tensors_ms_first': 0.106,
 'allocate_tensors_ms_avg': 0.106,
 'allocate_tensors_ms_%': 34.528}

In [18]:
def get_profiling_stats_cpu(filepath):
    """
    Parse "bechmarking.txt" file that has been generated from a .tflite model with TensorFlow benchmarking tool.  

    The native benchmark binary for linux is available from this page:  
    [https://www.tensorflow.org/lite/performance/measurement](https://www.tensorflow.org/lite/performance/measurement)   

    Args:
        filepath (pathlib.Path): filepath to the extracted .txt file

    Returns:
        dict: dictionary with extracted model summary profiling information 
    """

    lines = clean_model_summary(filepath)
    model_profiling, operator_profiling = get_profiling_lines(lines)
    profiling_stats = get_model_profiling_stats(model_profiling)
    return profiling_stats


In [19]:
prof_stats = get_profiling_stats_cpu(models_benchmark_path)
prof_stats

{'model_size_MB': 0.08908,
 'init_us': 95124,
 'first_inference_us': 5653,
 'warmup_avg_us': 663.688,
 'inference_avg_us': 696.187,
 'initialization_ms': 95.124,
 'modify_graph_with_delegate_ms_first': 0.201,
 'modify_graph_with_delegate_ms_avg': 0.201,
 'modify_graph_with_delegate_ms_%': 65.472,
 'modify_graph_with_delegate_mem_KB': 0.0,
 'allocate_tensors_ms_first': 0.106,
 'allocate_tensors_ms_avg': 0.106,
 'allocate_tensors_ms_%': 34.528}

# Operator profiling

In [20]:
def get_operator_df(text, name=""):
    df = pd.DataFrame(text)
    df.rename(columns=df.iloc[0, :], inplace=True) 
    df.drop(df.index[0], inplace=True)
    df.columns = clean_column_names(df)
    # try:
    df["%"] = df["%"].apply(string_percent_to_float)
    # except:
    #     df["avg %"] = df["avg %"].apply(string_percent_to_float)
    # try:
    df["cdf%"] = df["cdf%"].apply(string_percent_to_float)
    # except:
    #     df["cdf %"] = df["cdf %"].apply(string_percent_to_float)
    df["first"] = df["first"].map(float)
    df["avg ms"] = df["avg ms"].map(float)
    df["mem KB"] = df["mem KB"].map(float)
    df["times called"] = df["times called"].map(int)#
    #df.Name = name

    return df

In [21]:
def get_node_df(text, name=""):
    df = pd.DataFrame(text)
    df.rename(columns=df.iloc[0, :], inplace=True) 
    df.drop(df.index[0], inplace=True)
    df.columns = clean_column_names(df)
    df["avg %"] = df["avg %"].apply(string_percent_to_float)
    df["cdf %"] = df["cdf %"].apply(string_percent_to_float)
    df["avg ms"] = df["avg ms"].map(float)
    df["mem KB"] = df["mem KB"].map(float)
    df["times called"] = df["times called"].map(int)
    df["count"] = df["count"].map(int)
    df.Name = name

    return df

In [22]:
def get_index_of_operator_profiling_lines(lines):
    operator_lines_dict = {}
    for i, line in enumerate(lines):
        #if line.str.contains('= Run Order ='):
        if "= Run Order =" in line:
            operator_lines_dict["run_order"] = i
        elif "= Top by Computation Time =" in line:
            operator_lines_dict["top_by_computation_time"] = i
        elif "= Summary by node type =" in line:
            operator_lines_dict["summary_by_node_type"] = i

    # print(operator_lines_dict)
    return operator_lines_dict

In [23]:
# operator_lines_dict = get_index_of_operator_profiling_lines(operator_profiling)
# operator_lines_dict

In [24]:
def get_summary_by_node_type_df(operator_profiling, operator_lines_dict, model_name):
    summary_by_node_type = operator_profiling[operator_lines_dict["summary_by_node_type"] +1:-3]
    summary_by_node_type = remove_tabs(summary_by_node_type)
    summary_by_node_type_df = get_node_df(summary_by_node_type, name=f"Summary by node type - {model_name}")
    return summary_by_node_type_df

In [25]:
def get_operator_top_by_comp_time_df(operator_profiling, operator_lines_dict, model_name):

    top_by_comp_time = operator_profiling[
        operator_lines_dict["top_by_computation_time"]+ 1 : operator_lines_dict["summary_by_node_type"]- 1]
    top_by_comp_time_clean = remove_tabs(top_by_comp_time)

    df = pd.DataFrame(top_by_comp_time_clean)

    df.rename(columns=df.iloc[0, :], inplace=True)
    df.drop(df.index[0], inplace=True)
    df.columns = clean_column_names(df)

    df["%"] = df["%"].apply(string_percent_to_float)
    df["cdf%"] = df["cdf%"].apply(string_percent_to_float)
    df["first"] = df["first"].map(float)
    df["avg ms"] = df["avg ms"].map(float)
    df["mem KB"] = df["mem KB"].map(float)
    df["times called"] = df["times called"].map(int)

    return df

In [26]:
def get_run_order_df(operator_lines_dict, operator_profiling, model_name):
    run_order = operator_profiling[operator_lines_dict["run_order"] +1: operator_lines_dict["top_by_computation_time"]]
    run_order = remove_tabs(run_order)
    df = get_operator_df(run_order, name= f"Run order - {model_name}")
    return df

In [27]:
def get_profiling_dataframes_cpu(filepath, model_name):
    """
    Parse "bechmarking.txt" file that has been generated from a .tflite model with TensorFlow benchmarking tool.  

    The native benchmark binary for linux is available from this page:  
    [https://www.tensorflow.org/lite/performance/measurement](https://www.tensorflow.org/lite/performance/measurement)   

    Args:
        filepath (pathlib.Path): filepath to the extracted .txt file

    Returns:
        list(pd.DataFrame, pd.DataFrame, pd.DataFrame): tuple containing the following dataframes:

            operator_run_order_df, operator_by_comp_time_df, summary_node_type_df
    """

    lines = clean_model_summary(filepath)
    model_profiling, operator_profiling = get_profiling_lines(lines)
    operator_lines_dict = get_index_of_operator_profiling_lines(operator_profiling)

    operator_run_order_df = get_run_order_df(operator_lines_dict, operator_profiling, model_name)
    operator_by_comp_time_df = get_operator_top_by_comp_time_df(operator_profiling, operator_lines_dict, model_name)
    summary_node_type_df = get_summary_by_node_type_df(operator_profiling, operator_lines_dict, model_name)

    return operator_run_order_df, operator_by_comp_time_df, summary_node_type_df

In [28]:
operator_run_order_df, operator_by_comp_time_df, summary_node_type_df = get_profiling_dataframes_cpu(models_benchmark_path, model_name)

In [29]:
operator_run_order_df

,node type,first,avg ms,%,cdf%,mem KB,times called,Name
1,QUANTIZE,0.093,0.098,14.484,14.484,0.0,1,[tfl.quantize]:0
2,CONV_2D,0.090,0.083,12.362,26.845,0.0,1,[mobilenetv1/re_lu/Relu;mobilenetv1/batch_normalization/FusedBatchNormV3;mobilenetv1/batch_normalization_1/FusedBatchNormV3/ReadVariableOp;mobilenetv1/conv2d/BiasAdd;mobilenetv1/batch_normalization_1/FusedBatchNormV3;mobilenetv1/conv2d/Conv2D]:1
3,DEPTHWISE_CONV_2D,0.103,0.107,15.916,42.762,0.0,1,[mobilenetv1/re_lu_1/Relu;mobilenetv1/batch_normalization_1/FusedBatchNormV3;mobilenetv1/batch_normalization_1/FusedBatchNormV3/ReadVariableOp;mobilenetv1/depthwise_conv2d/depthwise;mobilenetv1/depthwise_conv2d/BiasAdd]:2
4,CONV_2D,0.040,0.042,6.210,48.972,0.0,1,[mobilenetv1/re_lu_2/Relu;mobilenetv1/batch_normalization_2/FusedBatchNormV3;mobilenetv1/batch_normalization_2/FusedBatchNormV3/ReadVariableOp;mobilenetv1/conv2d_1/BiasAdd;mobilenetv1/batch_normalization_3/FusedBatchNormV3;mobilenetv1/conv2d_1/Conv2D]:3
5,DEPTHWISE_CONV_2D,0.044,0.046,6.876,55.848,0.0,1,[mobilenetv1/re_lu_3/Relu;mobilenetv1/batch_normalization_3/FusedBatchNormV3;mobilenetv1/batch_normalization_2/FusedBatchNormV3/ReadVariableOp;mobilenetv1/depthwise_conv2d_1/depthwise;mobilenetv1/depthwise_conv2d_1/BiasAdd]:4
6,CONV_2D,0.015,0.015,2.295,58.144,0.0,1,[mobilenetv1/re_lu_4/Relu;mobilenetv1/batch_normalization_4/FusedBatchNormV3;mobilenetv1/batch_normalization_4/FusedBatchNormV3/ReadVariableOp;mobilenetv1/conv2d_2/BiasAdd;mobilenetv1/batch_normalization_7/FusedBatchNormV3;mobilenetv1/conv2d_2/Conv2D]:5
7,DEPTHWISE_CONV_2D,0.045,0.046,6.888,65.032,0.0,1,[mobilenetv1/re_lu_5/Relu;mobilenetv1/batch_normalization_5/FusedBatchNormV3;mobilenetv1/batch_normalization_4/FusedBatchNormV3/ReadVariableOp;mobilenetv1/depthwise_conv2d_2/depthwise;mobilenetv1/depthwise_conv2d_2/BiasAdd;mobilenetv1/batch_normalization_7/FusedBatchNormV3]:6
8,CONV_2D,0.016,0.017,2.454,67.485,0.0,1,[mobilenetv1/re_lu_6/Relu;mobilenetv1/batch_normalization_6/FusedBatchNormV3;mobilenetv1/batch_normalization_4/FusedBatchNormV3/ReadVariableOp;mobilenetv1/conv2d_3/BiasAdd;mobilenetv1/batch_normalization_7/FusedBatchNormV3;mobilenetv1/conv2d_3/Conv2D]:7
9,DEPTHWISE_CONV_2D,0.017,0.018,2.701,70.186,0.0,1,[mobilenetv1/re_lu_7/Relu;mobilenetv1/batch_normalization_7/FusedBatchNormV3;mobilenetv1/batch_normalization_4/FusedBatchNormV3/ReadVariableOp;mobilenetv1/depthwise_conv2d_3/depthwise;mobilenetv1/depthwise_conv2d_3/BiasAdd]:8
10,CONV_2D,0.006,0.007,1.083,71.269,0.0,1,[mobilenetv1/re_lu_8/Relu;mobilenetv1/batch_normalization_8/FusedBatchNormV3;mobilenetv1/batch_normalization_10/FusedBatchNormV3/ReadVariableOp;mobilenetv1/conv2d_4/BiasAdd;mobilenetv1/batch_normalization_11/FusedBatchNormV3;mobilenetv1/conv2d_4/Conv2D]:9


In [30]:
operator_by_comp_time_df

,node type,first,avg ms,%,cdf%,mem KB,times called,Name
1,DEPTHWISE_CONV_2D,0.103,0.107,15.916,15.916,0.0,1,[mobilenetv1/re_lu_1/Relu;mobilenetv1/batch_normalization_1/FusedBatchNormV3;mobilenetv1/batch_normalization_1/FusedBatchNormV3/ReadVariableOp;mobilenetv1/depthwise_conv2d/depthwise;mobilenetv1/depthwise_conv2d/BiasAdd]:2
2,QUANTIZE,0.093,0.098,14.484,30.400,0.0,1,[tfl.quantize]:0
3,CONV_2D,0.090,0.083,12.362,42.762,0.0,1,[mobilenetv1/re_lu/Relu;mobilenetv1/batch_normalization/FusedBatchNormV3;mobilenetv1/batch_normalization_1/FusedBatchNormV3/ReadVariableOp;mobilenetv1/conv2d/BiasAdd;mobilenetv1/batch_normalization_1/FusedBatchNormV3;mobilenetv1/conv2d/Conv2D]:1
4,DEPTHWISE_CONV_2D,0.045,0.046,6.888,49.650,0.0,1,[mobilenetv1/re_lu_5/Relu;mobilenetv1/batch_normalization_5/FusedBatchNormV3;mobilenetv1/batch_normalization_4/FusedBatchNormV3/ReadVariableOp;mobilenetv1/depthwise_conv2d_2/depthwise;mobilenetv1/depthwise_conv2d_2/BiasAdd;mobilenetv1/batch_normalization_7/FusedBatchNormV3]:6
5,DEPTHWISE_CONV_2D,0.044,0.046,6.876,56.526,0.0,1,[mobilenetv1/re_lu_3/Relu;mobilenetv1/batch_normalization_3/FusedBatchNormV3;mobilenetv1/batch_normalization_2/FusedBatchNormV3/ReadVariableOp;mobilenetv1/depthwise_conv2d_1/depthwise;mobilenetv1/depthwise_conv2d_1/BiasAdd]:4
6,CONV_2D,0.040,0.042,6.210,62.736,0.0,1,[mobilenetv1/re_lu_2/Relu;mobilenetv1/batch_normalization_2/FusedBatchNormV3;mobilenetv1/batch_normalization_2/FusedBatchNormV3/ReadVariableOp;mobilenetv1/conv2d_1/BiasAdd;mobilenetv1/batch_normalization_3/FusedBatchNormV3;mobilenetv1/conv2d_1/Conv2D]:3
7,DEPTHWISE_CONV_2D,0.026,0.028,4.161,66.897,0.0,1,[mobilenetv1/re_lu_9/Relu;mobilenetv1/batch_normalization_9/FusedBatchNormV3;mobilenetv1/batch_normalization_10/FusedBatchNormV3/ReadVariableOp;mobilenetv1/depthwise_conv2d_4/depthwise;mobilenetv1/depthwise_conv2d_4/BiasAdd;mobilenetv1/batch_normalization_11/FusedBatchNormV3]:10
8,DEPTHWISE_CONV_2D,0.017,0.018,2.701,69.598,0.0,1,[mobilenetv1/re_lu_7/Relu;mobilenetv1/batch_normalization_7/FusedBatchNormV3;mobilenetv1/batch_normalization_4/FusedBatchNormV3/ReadVariableOp;mobilenetv1/depthwise_conv2d_3/depthwise;mobilenetv1/depthwise_conv2d_3/BiasAdd]:8
9,CONV_2D,0.016,0.017,2.454,72.052,0.0,1,[mobilenetv1/re_lu_6/Relu;mobilenetv1/batch_normalization_6/FusedBatchNormV3;mobilenetv1/batch_normalization_4/FusedBatchNormV3/ReadVariableOp;mobilenetv1/conv2d_3/BiasAdd;mobilenetv1/batch_normalization_7/FusedBatchNormV3;mobilenetv1/conv2d_3/Conv2D]:7
10,CONV_2D,0.015,0.015,2.295,74.348,0.0,1,[mobilenetv1/re_lu_4/Relu;mobilenetv1/batch_normalization_4/FusedBatchNormV3;mobilenetv1/batch_normalization_4/FusedBatchNormV3/ReadVariableOp;mobilenetv1/conv2d_2/BiasAdd;mobilenetv1/batch_normalization_7/FusedBatchNormV3;mobilenetv1/conv2d_2/Conv2D]:5


In [31]:
summary_node_type_df

,Node type,count,avg ms,avg %,cdf %,mem KB,times called
1,DEPTHWISE_CONV_2D,13,0.327,50.000,50.000,0.0,13
2,CONV_2D,14,0.226,34.557,84.557,0.0,14
3,QUANTIZE,2,0.097,14.832,99.388,0.0,2
4,REDUCE_PROD,2,0.001,0.153,99.541,0.0,2
5,FULLY_CONNECTED,1,0.001,0.153,99.694,0.0,1
6,CONCATENATION,1,0.001,0.153,99.847,0.0,1
7,AVERAGE_POOL_2D,1,0.001,0.153,100.000,0.0,1
8,SOFTMAX,1,0.000,0.000,100.000,0.0,1
9,SHAPE,1,0.000,0.000,100.000,0.0,1
10,RESHAPE,2,0.000,0.000,100.000,0.0,2


In [32]:
# operator_profiling = lines[split_line+1:]
# operator_profiling

In [33]:
# operator_lines_dict = {}
# for i, line in enumerate(operator_profiling):
#     #if line.str.contains('= Run Order ='):
#     if "= Run Order =" in line:
#         operator_lines_dict["run_order"] = i
#     elif "= Top by Computation Time =" in line:
#         operator_lines_dict["top_by_computation_time"] = i
#     elif "= Summary by node type =" in line:
#         operator_lines_dict["summary_by_node_type"] = i

# operator_lines_dict

In [34]:
# summary_by_node_type = operator_profiling[operator_lines_dict["summary_by_node_type"] +1:-3]
# summary_by_node_type = remove_tabs(summary_by_node_type)
# summary_by_node_type

In [35]:
# summary_by_node_type_df = get_node_df(summary_by_node_type, name=f"Summary by node type - {model_name}")
# summary_by_node_type_df

In [36]:
#summary_by_node_type_df.Name

In [37]:
#summary_by_node_type_df.info

In [38]:
# top_by_comp_time = operator_profiling[operator_lines_dict["top_by_computation_time"] +1: operator_lines_dict["summary_by_node_type"]-1]
# top_by_comp_time = remove_tabs(top_by_comp_time)
# top_by_comp_time

In [39]:
# df_operator_top_by_comp_time = get_operator_df(top_by_comp_time, name=f"Top by compute time - {model_name}")
# df_operator_top_by_comp_time

In [40]:
# run_order = operator_profiling[operator_lines_dict["run_order"] +1: operator_lines_dict["top_by_computation_time"]]
# run_order = remove_tabs(run_order)
# #run_order

In [41]:
# df_operator_run_order = get_operator_df(run_order, name= f"Run order - {model_name}")
# #df_operator_run_order.style.set_properties(**{'text-align': 'left'})
# df_operator_run_order.head(20)

In [42]:
# df_operator_run_order.info()

# Peak memory usage

# Using tflite-tools for extracting tflite model data

In [43]:
models_peak_memory_path = models_dir.joinpath(model_name, f"{model_name}_peak_memory")
model_peak_memory_schedule = models_dir.joinpath(model_name, f"{model_name}_peak_memory", "execution_schedule_info.csv")
peak_memory_tensor_details = models_dir.joinpath(model_name, f"{model_name}_peak_memory", "tensor_details.csv")
models_peak_memory_path

WindowsPath('i:/tinyml/tiny_cnn/models/mobilenetv1_0.1_96_c3_o2_l5.MV1/mobilenetv1_0.1_96_c3_o2_l5.MV1_peak_memory')

In [44]:
models_tflite_opt_path

WindowsPath('i:/tinyml/tiny_cnn/models/mobilenetv1_0.1_96_c3_o2_l5.MV1/mobilenetv1_0.1_96_c3_o2_l5.MV1_INT8.tflite')

In [45]:
# ! python i:\\tinyml\\tflite-tools\\tflite_tools.py \
#     -i $models_tflite_opt_path \
#     --csv $models_peak_memory_path 
 

In [46]:
! python i:\tinyml\tflite-tools\tflite_tools.py -i $models_tflite_opt_path --csv $models_peak_memory_path

Writing model analysis to i:\tinyml\tiny_cnn\models\mobilenetv1_0.1_96_c3_o2_l5.MV1\mobilenetv1_0.1_96_c3_o2_l5.MV1_peak_memory in CSV format


In [47]:
def format_shape(x):
    x = str(x)
    x = x.strip()
    split_strings = x.split()
    joined_string = ", ".join(split_strings)
    #print(joined_string)
    joined_string = "(" + joined_string +")"
    return joined_string                                 

In [48]:
tensor_details_df = pd.read_csv(peak_memory_tensor_details)
#tensor_details_df

In [49]:
# tensor_details_df["Shape"].fillna(0, inplace=True)
# tensor_details_df["Shape"].unique()

In [50]:
# tensor_details_df["Shape"] = tensor_details_df["Shape"].apply(format_shape)
# tensor_details_df

In [51]:
def split_operator_col(x):
    x = str(x).strip("()")
    split_strings = x.split(";")
    #print(split_strings)
    return split_strings[-1].strip()

In [52]:
def get_middle_part(x):
    x = str(x).strip("()")
    split_strings = x.split("/")
    if len(split_strings) == 3:
        return split_strings[1]
    elif len(split_strings) ==4:
        return split_strings[1]
    else:
        return x

In [53]:
def clean_tensor_memory_df(df):
    
    df["name"] = df["Name"].apply(split_operator_col)
    df["Size"] = df["Size"].map(int)
    df["Name"] = df["Name"].apply(get_middle_part)
    
    df["Shape"].fillna(0, inplace=True)
    df["Shape"] = df["Shape"].apply(format_shape)

    # reorder columns
    df.insert(1, "name", df.pop("name"))
    df.insert(4, "Name", df.pop("Name"))

    # rename columns
    df.columns = ["id", "name", "shape", "size_b", "name_long"]

    return df


In [54]:
def get_tensor_details_df(filepath):
    print(f"Reading in {filepath}")
    df = pd.read_csv(file)
    print("Cleaning up the dataframe.")
    df = clean_tensor_memory_df(df)
    return df


In [55]:

my_df = get_tensor_details_df(peak_memory_tensor_details)
my_df

Reading in i:\tinyml\tiny_cnn\models\mobilenetv1_0.1_96_c3_o2_l5.MV1\mobilenetv1_0.1_96_c3_o2_l5.MV1_peak_memory\tensor_details.csv


NameError: name 'file' is not defined

In [ ]:
clean_tensor_df = clean_tensor_memory_df(tensor_details_df)
clean_tensor_df 

,id,name,shape,size_b,name_long
0,0,serving_default_input_1:0,"(1, 96, 96, 3)",27648,serving_default_input_1:0
1,49,tfl.quantize,"(1, 96, 96, 3)",27648,tfl.quantize
2,50,mobilenetv1/conv2d/Conv2D1,"(1, 48, 48, 3)",6912,mobilenetv1/re_lu/Relu;mobilenetv1/batch_normalization/FusedBatchNormV3;mobilenetv1/conv2d/BiasAdd/ReadVariableOp;mobilenetv1/conv2d/BiasAdd;mobilenetv1/conv2d_11/Conv2D;mobilenetv1/conv2d/Conv2D1
3,51,mobilenetv1/conv2d_11/Conv2D1,"(1, 48, 48, 3)",6912,mobilenetv1/re_lu_1/Relu;mobilenetv1/batch_normalization_1/FusedBatchNormV3;mobilenetv1/depthwise_conv2d/depthwise;mobilenetv1/depthwise_conv2d/BiasAdd/ReadVariableOp;mobilenetv1/depthwise_conv2d/BiasAdd;mobilenetv1/conv2d_11/Conv2D1
4,52,mobilenetv1/conv2d_1/Conv2D1,"(1, 48, 48, 6)",13824,mobilenetv1/re_lu_2/Relu;mobilenetv1/batch_normalization_2/FusedBatchNormV3;mobilenetv1/conv2d_1/BiasAdd/ReadVariableOp;mobilenetv1/conv2d_1/BiasAdd;mobilenetv1/depthwise_conv2d_1/depthwise;mobilenetv1/conv2d_1/Conv2D1
5,53,mobilenetv1/depthwise_conv2d_1/BiasAdd1,"(1, 24, 24, 6)",3456,mobilenetv1/re_lu_3/Relu;mobilenetv1/batch_normalization_3/FusedBatchNormV3;mobilenetv1/depthwise_conv2d_1/depthwise;mobilenetv1/depthwise_conv2d_1/BiasAdd/ReadVariableOp;mobilenetv1/depthwise_conv2d_1/BiasAdd1
6,54,mobilenetv1/conv2d_2/Conv2D1,"(1, 24, 24, 12)",6912,mobilenetv1/re_lu_4/Relu;mobilenetv1/batch_normalization_4/FusedBatchNormV3;mobilenetv1/conv2d_2/BiasAdd/ReadVariableOp;mobilenetv1/conv2d_2/BiasAdd;mobilenetv1/depthwise_conv2d_3/depthwise;mobilenetv1/conv2d_2/Conv2D1
7,55,mobilenetv1/depthwise_conv2d_3/depthwise1,"(1, 24, 24, 12)",6912,mobilenetv1/re_lu_5/Relu;mobilenetv1/batch_normalization_5/FusedBatchNormV3;mobilenetv1/depthwise_conv2d_2/depthwise;mobilenetv1/depthwise_conv2d_2/BiasAdd/ReadVariableOp;mobilenetv1/depthwise_conv2d_2/BiasAdd;mobilenetv1/depthwise_conv2d_3/depthwise1
8,56,mobilenetv1/conv2d_3/Conv2D1,"(1, 24, 24, 12)",6912,mobilenetv1/re_lu_6/Relu;mobilenetv1/batch_normalization_6/FusedBatchNormV3;mobilenetv1/conv2d_3/BiasAdd/ReadVariableOp;mobilenetv1/conv2d_3/BiasAdd;mobilenetv1/depthwise_conv2d_3/depthwise;mobilenetv1/conv2d_3/Conv2D1
9,57,mobilenetv1/depthwise_conv2d_3/BiasAdd1,"(1, 12, 12, 12)",1728,mobilenetv1/re_lu_7/Relu;mobilenetv1/batch_normalization_7/FusedBatchNormV3;mobilenetv1/depthwise_conv2d_3/depthwise;mobilenetv1/depthwise_conv2d_3/BiasAdd/ReadVariableOp;mobilenetv1/depthwise_conv2d_3/BiasAdd1


# Peak memory df

In [ ]:
peak_memory_df = pd.read_csv(model_peak_memory_schedule)
peak_memory_df.head()

,Operator,Working set,Memory use
0,tfl.quantize,0 49,55296
1,mobilenetv1/re_lu/Relu;mobilenetv1/batch_normalization/FusedBatchNormV3;mobilenetv1/conv2d/BiasAdd/ReadVariableOp;mobilenetv1/conv2d/BiasAdd;mobilenetv1/conv2d_11/Conv2D;mobilenetv1/conv2d/Conv2D1,49 50,34560
2,mobilenetv1/re_lu_1/Relu;mobilenetv1/batch_normalization_1/FusedBatchNormV3;mobilenetv1/depthwise_conv2d/depthwise;mobilenetv1/depthwise_conv2d/BiasAdd/ReadVariableOp;mobilenetv1/depthwise_conv2d/BiasAdd;mobilenetv1/conv2d_11/Conv2D1,50 51,13824
3,mobilenetv1/re_lu_2/Relu;mobilenetv1/batch_normalization_2/FusedBatchNormV3;mobilenetv1/conv2d_1/BiasAdd/ReadVariableOp;mobilenetv1/conv2d_1/BiasAdd;mobilenetv1/depthwise_conv2d_1/depthwise;mobilenetv1/conv2d_1/Conv2D1,51 52,20736
4,mobilenetv1/re_lu_3/Relu;mobilenetv1/batch_normalization_3/FusedBatchNormV3;mobilenetv1/depthwise_conv2d_1/depthwise;mobilenetv1/depthwise_conv2d_1/BiasAdd/ReadVariableOp;mobilenetv1/depthwise_conv2d_1/BiasAdd1,52 53,17280


In [ ]:
peak_memory_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Operator     30 non-null     object
 1   Working set  30 non-null     object
 2   Memory use   30 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 848.0+ bytes


In [ ]:
# peak_memory_df["Name"] = peak_memory_df["Operator"].apply(split_operator_col)
# peak_memory_df["Memory use"] = peak_memory_df["Memory use"].map(int)

In [ ]:
# peak_memory_df["Name"] = peak_memory_df["Name"].apply(get_middle_part)

In [ ]:
peak_memory_df.columns

Index(['Operator', 'Working set', 'Memory use'], dtype='object')

In [ ]:
# peak_memory_df.insert(0, "Name", peak_memory_df.pop("Name"))
# peak_memory_df.insert(3, "Operator", peak_memory_df.pop("Operator"))

In [ ]:
def clean_working_set(x):
    split_strings = x.split()
    if len(split_strings) == 2:
        return (int(split_strings[0]),int(split_strings[1]))
    elif len(split_strings) == 3:
        return (int(split_strings[0]),int(split_strings[1]),int(split_strings[2]))
    else:
        return x

In [ ]:
# peak_memory_df["Working set"] = peak_memory_df["Working set"].apply(clean_working_set)

In [ ]:
# peak_memory_df.columns = ["name", "tensor_IDs", "RAM_b", "operator"]

In [ ]:
def clean_peak_memory_df(df):
    
    df["Name"] = df["Operator"].apply(split_operator_col)
    df["Memory use"] = df["Memory use"].map(int)
    df["Name"] = df["Name"].apply(get_middle_part)
    df["Working set"] = df["Working set"].apply(clean_working_set)

    # reorder columns
    df.insert(0, "Name", df.pop("Name"))
    df.insert(3, "Operator", df.pop("Operator"))

    # rename columns
    df.columns = ["name", "tensor_IDs", "RAM_b", "operator"]

    return df


In [ ]:
def get_peak_memory_df(filepath):
    print(f"Reading in {filepath}")
    df = pd.read_csv(filepath)
    print("Cleaning up the dataframe.")
    df = clean_peak_memory_df(df)
    return df

In [ ]:
new_peak_df = get_peak_memory_df(model_peak_memory_schedule)
new_peak_df

Reading in i:\tinyml\tiny_cnn\models\mobilenetv1_0.1_96_c3_o3_l2\mobilenetv1_0.1_96_c3_o3_l2_peak_memory\execution_schedule_info.csv
Cleaning up the dataframe.


,name,tensor_IDs,RAM_b,operator
0,tfl.quantize,"(0, 49)",55296,tfl.quantize
1,conv2d,"(49, 50)",34560,mobilenetv1/re_lu/Relu;mobilenetv1/batch_normalization/FusedBatchNormV3;mobilenetv1/conv2d/BiasAdd/ReadVariableOp;mobilenetv1/conv2d/BiasAdd;mobilenetv1/conv2d_11/Conv2D;mobilenetv1/conv2d/Conv2D1
2,conv2d_11,"(50, 51)",13824,mobilenetv1/re_lu_1/Relu;mobilenetv1/batch_normalization_1/FusedBatchNormV3;mobilenetv1/depthwise_conv2d/depthwise;mobilenetv1/depthwise_conv2d/BiasAdd/ReadVariableOp;mobilenetv1/depthwise_conv2d/BiasAdd;mobilenetv1/conv2d_11/Conv2D1
3,conv2d_1,"(51, 52)",20736,mobilenetv1/re_lu_2/Relu;mobilenetv1/batch_normalization_2/FusedBatchNormV3;mobilenetv1/conv2d_1/BiasAdd/ReadVariableOp;mobilenetv1/conv2d_1/BiasAdd;mobilenetv1/depthwise_conv2d_1/depthwise;mobilenetv1/conv2d_1/Conv2D1
4,depthwise_conv2d_1,"(52, 53)",17280,mobilenetv1/re_lu_3/Relu;mobilenetv1/batch_normalization_3/FusedBatchNormV3;mobilenetv1/depthwise_conv2d_1/depthwise;mobilenetv1/depthwise_conv2d_1/BiasAdd/ReadVariableOp;mobilenetv1/depthwise_conv2d_1/BiasAdd1
5,conv2d_2,"(53, 54)",10368,mobilenetv1/re_lu_4/Relu;mobilenetv1/batch_normalization_4/FusedBatchNormV3;mobilenetv1/conv2d_2/BiasAdd/ReadVariableOp;mobilenetv1/conv2d_2/BiasAdd;mobilenetv1/depthwise_conv2d_3/depthwise;mobilenetv1/conv2d_2/Conv2D1
6,depthwise_conv2d_3,"(54, 55)",13824,mobilenetv1/re_lu_5/Relu;mobilenetv1/batch_normalization_5/FusedBatchNormV3;mobilenetv1/depthwise_conv2d_2/depthwise;mobilenetv1/depthwise_conv2d_2/BiasAdd/ReadVariableOp;mobilenetv1/depthwise_conv2d_2/BiasAdd;mobilenetv1/depthwise_conv2d_3/depthwise1
7,conv2d_3,"(55, 56)",13824,mobilenetv1/re_lu_6/Relu;mobilenetv1/batch_normalization_6/FusedBatchNormV3;mobilenetv1/conv2d_3/BiasAdd/ReadVariableOp;mobilenetv1/conv2d_3/BiasAdd;mobilenetv1/depthwise_conv2d_3/depthwise;mobilenetv1/conv2d_3/Conv2D1
8,depthwise_conv2d_3,"(56, 57)",8640,mobilenetv1/re_lu_7/Relu;mobilenetv1/batch_normalization_7/FusedBatchNormV3;mobilenetv1/depthwise_conv2d_3/depthwise;mobilenetv1/depthwise_conv2d_3/BiasAdd/ReadVariableOp;mobilenetv1/depthwise_conv2d_3/BiasAdd1
9,conv2d_4,"(57, 58)",5328,mobilenetv1/re_lu_8/Relu;mobilenetv1/batch_normalization_8/FusedBatchNormV3;mobilenetv1/conv2d_4/BiasAdd/ReadVariableOp;mobilenetv1/conv2d_4/BiasAdd;mobilenetv1/depthwise_conv2d_5/depthwise;mobilenetv1/conv2d_4/Conv2D1


In [ ]:
peak_memory_df

,Operator,Working set,Memory use
0,tfl.quantize,0 49,55296
1,mobilenetv1/re_lu/Relu;mobilenetv1/batch_normalization/FusedBatchNormV3;mobilenetv1/conv2d/BiasAdd/ReadVariableOp;mobilenetv1/conv2d/BiasAdd;mobilenetv1/conv2d_11/Conv2D;mobilenetv1/conv2d/Conv2D1,49 50,34560
2,mobilenetv1/re_lu_1/Relu;mobilenetv1/batch_normalization_1/FusedBatchNormV3;mobilenetv1/depthwise_conv2d/depthwise;mobilenetv1/depthwise_conv2d/BiasAdd/ReadVariableOp;mobilenetv1/depthwise_conv2d/BiasAdd;mobilenetv1/conv2d_11/Conv2D1,50 51,13824
3,mobilenetv1/re_lu_2/Relu;mobilenetv1/batch_normalization_2/FusedBatchNormV3;mobilenetv1/conv2d_1/BiasAdd/ReadVariableOp;mobilenetv1/conv2d_1/BiasAdd;mobilenetv1/depthwise_conv2d_1/depthwise;mobilenetv1/conv2d_1/Conv2D1,51 52,20736
4,mobilenetv1/re_lu_3/Relu;mobilenetv1/batch_normalization_3/FusedBatchNormV3;mobilenetv1/depthwise_conv2d_1/depthwise;mobilenetv1/depthwise_conv2d_1/BiasAdd/ReadVariableOp;mobilenetv1/depthwise_conv2d_1/BiasAdd1,52 53,17280
5,mobilenetv1/re_lu_4/Relu;mobilenetv1/batch_normalization_4/FusedBatchNormV3;mobilenetv1/conv2d_2/BiasAdd/ReadVariableOp;mobilenetv1/conv2d_2/BiasAdd;mobilenetv1/depthwise_conv2d_3/depthwise;mobilenetv1/conv2d_2/Conv2D1,53 54,10368
6,mobilenetv1/re_lu_5/Relu;mobilenetv1/batch_normalization_5/FusedBatchNormV3;mobilenetv1/depthwise_conv2d_2/depthwise;mobilenetv1/depthwise_conv2d_2/BiasAdd/ReadVariableOp;mobilenetv1/depthwise_conv2d_2/BiasAdd;mobilenetv1/depthwise_conv2d_3/depthwise1,54 55,13824
7,mobilenetv1/re_lu_6/Relu;mobilenetv1/batch_normalization_6/FusedBatchNormV3;mobilenetv1/conv2d_3/BiasAdd/ReadVariableOp;mobilenetv1/conv2d_3/BiasAdd;mobilenetv1/depthwise_conv2d_3/depthwise;mobilenetv1/conv2d_3/Conv2D1,55 56,13824
8,mobilenetv1/re_lu_7/Relu;mobilenetv1/batch_normalization_7/FusedBatchNormV3;mobilenetv1/depthwise_conv2d_3/depthwise;mobilenetv1/depthwise_conv2d_3/BiasAdd/ReadVariableOp;mobilenetv1/depthwise_conv2d_3/BiasAdd1,56 57,8640
9,mobilenetv1/re_lu_8/Relu;mobilenetv1/batch_normalization_8/FusedBatchNormV3;mobilenetv1/conv2d_4/BiasAdd/ReadVariableOp;mobilenetv1/conv2d_4/BiasAdd;mobilenetv1/depthwise_conv2d_5/depthwise;mobilenetv1/conv2d_4/Conv2D1,57 58,5328


## Getting peak memory info from stdout

In [ ]:
# ! code $models_peak_memory_path

In [ ]:
# %%capture peak_memory_str

# ! python i:\tinyml\tflite-tools\tflite_tools.py -i $models_tflite_opt_path

In [ ]:
# x = peak_memory_str.stdout
# peak_memory_lines = x.split("\n")
# peak_memory_lines

In [ ]:
# peak_mem_lines_dict = {}
# for i, line in enumerate(peak_memory_lines):
#     #if line.str.contains('= Run Order ='):
#     if "|  Id | " in line:
#         peak_mem_lines_dict["header"] = i
#     elif "Current peak memory usage:" in line:
#         peak_mem_lines_dict["peak_memory"] = i
#     elif "Operator execution schedule:" in line:
#         peak_mem_lines_dict["operator_schedule"] = i

# peak_mem_lines_dict

NameError: name 'peak_memory_lines' is not defined

In [ ]:
operator_schedule = peak_memory_lines[peak_mem_lines_dict["operator_schedule"]: peak_mem_lines_dict["peak_memory"]]
operator_schedule

In [ ]:
operator_lines =[]
for line in operator_schedule:
    if line.startswith("|"):
        operator_lines.append(line.strip("|"))
#operator_lines

operator_data =[]
for row in operator_lines:
    operator_data_dict = {}
    data =row.split("|")
    #print(data)
    operator_data_dict["operator"] = data[0].strip()
    operator_data_dict["tensor_ids_in_memory"] = data[1].strip()
    operator_data_dict["peak_memory_kb"] = data[2].strip().replace(",", "")
    operator_data.append(operator_data_dict)

operator_data

In [ ]:
peak_memory_df = pd.DataFrame.from_dict(operator_data)
peak_memory_df = peak_memory_df.drop([0])
peak_memory_df["peak_memory_kb"] = peak_memory_df["peak_memory_kb"].map(float)/1000.0
peak_memory_df.Name = f"Peak memory - {model_name}"
peak_memory_df

In [ ]:
models_peak_memory_df_path = models_dir.joinpath(model_name, "peak_memory_df")
models_peak_memory_df_path

In [ ]:
peak_memory_df.to_pickle(models_peak_memory_df_path)

In [ ]:
peak_memory_df["peak_memory_kb"].max()

In [ ]:
tensor_information = peak_memory_lines[:peak_mem_lines_dict["operator_schedule"]]
tensor_information

In [ ]:
data_lines =[]
for line in tensor_information:
    if line.startswith("|"):
        data_lines.append(line.strip("|"))
#data_lines

split_data =[]
for row in data_lines:
    line_dict = {}
    data =row.split("|")
    #print(data)
    line_dict["tensor_id"] = data[0].strip()
    line_dict["tensor"] = data[1].strip()
    line_dict["shape"] = data[2].strip()
    line_dict["size_in_RAM_kb"] = data[3].strip().replace(",", "")
    split_data.append(line_dict)

split_data


In [ ]:
tensor_df = pd.DataFrame.from_dict(split_data)
tensor_df = tensor_df.drop([0])
tensor_df["size_in_RAM_kb"] = tensor_df["size_in_RAM_kb"].map(float)/1000.0
tensor_df.Name = f"Tensor information - {model_name}"
tensor_df

In [ ]:
tensor_df.Name

In [ ]:
tensor_info_df_path = models_dir.joinpath(model_name, "tensor_info_df")
tensor_info_df_path

In [ ]:
tensor_df.to_pickle(tensor_info_df_path)

## Parse peak memory

In [ ]:
peak_memory_line = peak_memory_lines[peak_mem_lines_dict["peak_memory"]]
peak_memory_line

In [ ]:
peak_memory_kb = float(peak_memory_line.split(":")[-1].strip().split()[0].replace(",", ".") )
peak_memory_kb

In [ ]:
model_stats["peak_memory_kb"] = peak_memory_kb

In [ ]:
model_stats

# Logging to wandb

In [ ]:
# Generate run ids
id = wandb.wandb.sdk.lib.runid.generate_id()

PROJECT = model_name.split("_")[0]

run = wandb.init(
        # Set the project where this run will be logged
        project=PROJECT, 
        name = model_name,
        id = id, 
        resume="allow",
        sync_tensorboard=True
        )
# Specify the configuration variables
config = wandb.config
config.architecture = model_name

# # Create a table
operator_top_by_time_table = wandb.Table(dataframe=df_operator_top_by_comp_time)
# operator_top_by_time_artifact = wandb.Artifact(name=f'{model_name}_operator_top_by_time', type='dataframe')
# operator_top_by_time_artifact.add(operator_top_by_time_table, "Profiling operators - top by time")
# run.log_artifact(operator_top_by_time_artifact)

summary_by_node_type_table = wandb.Table(dataframe=summary_by_node_type_df)
# summary_by_node_type_artifact = wandb.Artifact(name=f'{model_name}_summary_by_node_type', type='dataframe')
# summary_by_node_type_artifact.add(summary_by_node_type_table, "Node type summary")
# run.log_artifact(summary_by_node_type_artifact)

operator_run_order_table = wandb.Table(dataframe=df_operator_run_order)
# operator_run_order_artifact = wandb.Artifact(name=f'{model_name}_operator_run_order', type='dataframe')
# operator_run_order_artifact.add(operator_run_order_table, "Run order")
# run.log_artifact(operator_run_order_artifact)


run.log({"Profiling operators - top by time": operator_top_by_time_table})
run.log({"Node type summary" : summary_by_node_type_table})
run.log({"Run order" : operator_run_order_table})
# run.log({"Chart_table": table})

# table = wandb.Table(columns = ["plotly_figure"])

# # Create path for Plotly figure
# path_to_plotly_html = "./plotly_figure.html"

# # Write Plotly figure to HTML
# fig.write_html(path_to_plotly_html, auto_play = False) # Setting auto_play to False prevents animated Plotly charts from playing in the table automatically

# # Add Plotly figure as HTML file into Table
# table.add_data(wandb.Html(path_to_plotly_html))

# # Log Table
# run.log({"Chart_table": table})

wandb.log(model_stats)

# #wandb.log({"Peak activations chart": fig})

wandb.finish()